In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver 
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 2.5MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [96.1 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://developer.download.nvidia.com

In [ ]:
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json
import pprint

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get('https://www.subito.it/annunci-lombardia/affitto/camere-posti-letto/milano/milano/?q=Affitto&rt=1')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [ ]:
#funzione che estrae i dati dalle singole card 
detail_urls = []
id_urls = []
def subito_extraction(card):
  title = ""
  url = ""
  id = ""
  place_date = ""
  price = ""
  global detail_urls
  global id_urls

  try:
    list_url = card.find_elements_by_css_selector("a")
    if (len(list_url) > 0):
      url = list_url[0].get_attribute("href")
      detail_urls.append(url)
      splitted_url = detail_urls[-1].split(sep="-")
      id = splitted_url[-1]
      id_urls.append(id)
    else:
      print("Error: url not found")

    list_title = card.find_elements_by_css_selector("h2.jsx-3045029806.item-title")
    if (len(list_title) > 0):
      title = list_title[0].text
    else:
      print("Error: title not found")
    list_place_date = card.find_elements_by_css_selector("div.posting-time-place div.classes_container__xz7rh")
    if (len(list_place_date) > 0):
      place_date = list_place_date[0].text
    else:
      print("Error: place/date not found")
    list_price = card.find_elements_by_css_selector("h6.classes_price__HmHqw")
    if(len(list_price) > 0):
      price = list_price[0].text
    else:
      print("Error: price not found")   
  except:
    pass

  return {'Title':title,
                        'Url': url,
                        'Place and date': place_date,
                        'Price': price,
                        'Id': id
                        }

In [ ]:
info_rentMI = []

for pg in tqdm(range(1,16)):
  wd.get(f"https://www.subito.it/annunci-lombardia/affitto/camere-posti-letto/milano/milano/?q=Affitto&o={pg}&rt=1")
  list_rent = wd.find_elements_by_css_selector("div.jsx-3120895872.items__item")
  for card in list_rent:
    info_rentMI.append(subito_extraction(card))

print(len(info_rentMI))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found
Error: price not found

495


In [ ]:
ds_rentMI = pd.DataFrame(info_rentMI)
ds_rentMI.head(25)

,Title,Url,Place and date,Price,Id
0,Vicino MM Bisceglie stanza arredata luminosa,https://www.subito.it/camere-posti-letto/vicin...,Milano (MI) - Oggi alle 14:45,600 €,352103704.htm
1,Stanza matrimoniale a ragazza a Rogoredo,https://www.subito.it/camere-posti-letto/stanz...,Milano (MI) - Oggi alle 14:08,400 €,352100453.htm
2,Milano - Stanza singola ammobiliata,https://www.subito.it/camere-posti-letto/milan...,Milano (MI) - Oggi alle 13:08,450 €,352095609.htm
3,Camera singola,https://www.subito.it/camere-posti-letto/camer...,Milano (MI) - Oggi alle 12:53,400 €,352094169.htm
4,Stanza singola- Sant' Ambrogio,https://www.subito.it/camere-posti-letto/stanz...,Milano (MI) - Oggi alle 12:30,750 €,352091819.htm
5,Stanza singola in meraviglioso contesto,https://www.subito.it/camere-posti-letto/stanz...,Milano (MI),350 €,331487656.htm
6,Camera singola,https://www.subito.it/camere-posti-letto/camer...,Milano (MI),670 €,337888210.htm
7,Bicocca vicinanze Camera 580 doppia uso singola,https://www.subito.it/camere-posti-letto/bicoc...,Milano (MI),580 €,350659920.htm
8,Camera doppia uso singola S.Siro MM1 Bisceglie,https://www.subito.it/camere-posti-letto/camer...,Milano (MI) - Oggi alle 12:12,600 €,352090135.htm
9,Stanza singola terrazzino privato (M1 Primatic...,https://www.subito.it/camere-posti-letto/stanz...,Milano (MI) - Oggi alle 11:54,,352088353.htm


In [ ]:
ds_rentMI.to_csv('rent_mi_sept')

In [ ]:
#import pandas as pd
#df = pd.read_csv('rent_mi_senzaUrl')
#df.to_excel('rent_mi_senzaUrlEx.xlsx')

In [ ]:
#per capire come estrapolare l'id dall'url
for detail_url in detail_urls:
  splitted_url = detail_url.split(sep="-")
  x = splitted_url[-1]
  print(x)

348420464.htm
348056343.htm
347939494.htm
348388624.htm
348381265.htm
340894214.htm
345204625.htm
343114481.htm
343114481.htm
344407852.htm
348345187.htm
348344216.htm
348340206.htm
348309908.htm
348304133.htm
344105613.htm
348296396.htm
348275720.htm
348272117.htm
348267930.htm
348261102.htm
348245595.htm
348223020.htm
348218369.htm
348178064.htm
348168678.htm
331487656.htm
348095778.htm
348091634.htm
348074040.htm
348067369.htm
348058572.htm
348020416.htm
